In [1]:
import pandas as pd 
import numpy as np
import datetime 

# PART 1

In [ ]:

# reading excel_file
xlx_sheet = pd.ExcelFile('Analytics Position Case Study.xlsx') 

## Function to read different sheets and create columns 
def dataclean(sheet):
    in_dataf = pd.read_excel(xlx_sheet, sheet)
    #locating rows and columns in pandas data frame 
    new_header_xlx = in_dataf.iloc[2] #2nd index row intake 
    in_dataf = in_dataf[3:] #indexing for row
    
    in_dataf.columns = new_header_xlx
    
    in_dataf.reset_index(drop=True, inplace=True)
    
    in_dataf.rename_axis(None, axis=1, inplace=True)
    
    in_dataf.columns = in_dataf.columns.str.lower()
    
    # conversion of date time formart to datetime64 as per requirement 
    in_dataf['date_time'] = pd.to_datetime(in_dataf['datetime'])
    in_dataf['date_new'] = pd.to_datetime(in_dataf['date_time'].dt.date)
    in_dataf['session']= np.where(in_dataf['date_time'].dt.hour< 12, 'S1','S2')
    
    return in_dataf

dataf_dict ={'GAME_PLAYED':'User Gameplay data','MONEY_DEPOSIT':'Deposit Data','MONEY_WITHDRAWAL':'Withdrawal Data'}
# For key & values in dataf_dict.items():

print("DATA FRAME IS CREATED FROM GIVEN DATA (VINDIATA)")

for key,value in dataf_dict.items():
    exec(f'{key}_df = dataclean(value)')
    print(f'{key}')
    

# PART 2

In [18]:


GAME_PLAYED_df['date_time'].dt.hour

# Merging deposit and withdrawal sheets in pandas 
outside_dframe = pd.merge(MONEY_DEPOSIT_df, MONEY_WITHDRAWAL_df, on=('user id', 'date_new', 'session'), how='outer', suffixes=('_DEPOSIT', '_WITHDRAWAL'))

# Merging gameplay and outside data frame in pandas 
ff_dframe = pd.merge(outside_dframe, GAME_PLAYED_df, on=('user id', 'date_new', 'session'), how='outer', suffixes=('outside', 'GAME'))

# Dropping data not required in pandas (columns)
ff_dframe.drop(columns=['datetime_DEPOSIT', 'date_time_DEPOSIT', 'datetime_WITHDRAWAL', 'date_time_WITHDRAWAL', 'datetime', 'date_time'], inplace=True)

# Creating deposit, withrawal sum and count in pandas 
ff_dframe2 = ff_dframe.groupby(['user id', 'date_new', 'session']).agg(deposit_sum=('amount_DEPOSIT', 'sum'), deposit_count=('amount_DEPOSIT', 'count'), \
    withdrawal_sum=('amount_WITHDRAWAL', 'sum'), withdrawal_count=('amount_WITHDRAWAL', 'count'), games_played=('games played', 'sum')).reset_index()


# Using lambda function creating difference in withrawal and deposit count using Data Frame 
ff_dframe2['difference'] = ff_dframe2.apply(lambda x: x['deposit_count'] - x['withdrawal_count'] if (x['deposit_count'] - x['withdrawal_count']) >0 else 0, axis=1)

# Calculating final loyalty points of multiplayer game using data 
ff_dframe2['points'] = ff_dframe2['deposit_sum']*(0.01) + ff_dframe2['withdrawal_sum']*(0.005) + ff_dframe2['difference']*(0.001)\
     + ff_dframe2['games_played']*(0.2)

ff_dframe2['user id'] = ff_dframe2['user id'].astype(int)

# Add number of rows in head to print output for following
print(ff_dframe2.head(5))

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


   user id   date_new session deposit_sum  deposit_count withdrawal_sum  \
0        0 2022-04-10      S2           0              0              0   
1        0 2022-05-10      S2           0              0              0   
2        0 2022-06-10      S1           0              0              0   
3        0 2022-10-10      S1           0              0              0   
4        0 2022-10-10      S2           0              0              0   

   withdrawal_count games_played  difference points  
0                 0          1.0           0    0.2  
1                 0          1.0           0    0.2  
2                 0          1.0           0    0.2  
3                 0          1.0           0    0.2  
4                 0          1.0           0    0.2  


### 2nd Oct S1 

In [4]:

DFRAME1 = ff_dframe2[(ff_dframe2['date_new'] == '2022-02-10') & (ff_dframe2['session'] == 'S1')]
DFRAME1[['user id', 'points']]

,user id,points
23,2,0.4
131,5,55.809
193,6,80.402
234,7,1.0
273,8,1.2
...,...,...
38693,990,14.0
38785,992,8.4
38875,996,2.2
38937,997,1.0


### 16 Oct S2

In [19]:

DFRAME2 = ff_dframe2[(ff_dframe2['date_new'] == '2022-10-16') & (ff_dframe2['session'] == 'S2')]
DFRAME2[['user id', 'points']]

,user id,points
47,2,0.4
156,5,93.618
211,6,0.2
298,8,1.0
360,9,1030.251
...,...,...
38718,990,12.4
38767,991,60.201
38810,992,8.8
38900,996,1.8


### 18 Oct S1

In [20]:

DFRAME3 = ff_dframe2[(ff_dframe2['date_new'] == '2022-10-18') & (ff_dframe2['session'] == 'S1')]
DFRAME3[['user id', 'points']]

,user id,points
50,2,0.4
100,3,0.4
159,5,1.6
250,7,0.6
301,8,1.8
...,...,...
38721,990,14.4
38813,992,9.4
38903,996,41.005
38959,997,0.2


### 26 Oct S2

In [21]:

DFRAME4 = ff_dframe2[(ff_dframe2['date_new'] == '2022-10-26') & (ff_dframe2['session'] == 'S2')]
DFRAME4[['user id', 'points']]

,user id,points
67,2,900.002
176,5,40.804
223,6,0.2
260,7,33.285
318,8,1.4
...,...,...
38830,992,6608.8
38867,995,0.2
38920,996,0.8
38974,997,0.4


### loyalty program winner 

In [22]:
DFRAME5 = ff_dframe2.sort_values(by=['points', 'games_played'], ascending=False)
DFRAME5[['user id', 'points', 'games_played']]

,user id,points,games_played
13467,365,100636.2,168.0
33150,856,76820.903,303.0
25492,663,74242.612,212.0
13470,365,70140.21,114.0
33174,856,69832.6,188.0
...,...,...,...
39018,999,0.2,1.0
39026,999,0.2,1.0
39031,999,0.2,1.0
39036,999,0.2,1.0


### Average Deposit amount


In [24]:
DFRAME6 = ff_dframe2['deposit_sum'].mean().round(decimals=2)
print(f"Average Deposit Amount - {DFRAME6} Points")

Average Deposit Amount - 22809.23 Points


### Average deposit amount per user in month 


In [25]:
DFRAME7 = ff_dframe2['deposit_sum'].mean()/ff_dframe2['user id'].count()
print(f"Average Deposit Amount per user in a month - {DFRAME7} Points")

Average Deposit Amount per user in a month - 0.584088186050309 Points


### Average number of game played per user 


In [27]:
DFRAME8 = ff_dframe2['games_played'].mean().round(2)
print(f"Average number of games played per user - {DFRAME8}")

Average number of games played per user - 10.73


### Top 50 Winners 


In [12]:
TOP50_DFRAME = ff_dframe2.sort_values(by=['points', 'games_played'], ascending=False)
TOP50_DFRAME[['user id', 'points', 'games_played']].head(50)

,user id,points,games_played
13467,365,100636.2,168.0
33150,856,76820.903,303.0
25492,663,74242.612,212.0
13470,365,70140.21,114.0
33174,856,69832.6,188.0
6578,182,62953.2,108.0
13464,365,58051.53,186.0
38630,989,56397.0,110.0
13442,365,56058.86,52.0
38657,989,51310.854,54.0


### After calculating the loyalty points for the whole month find out which 50 players are at the top of the leaderboard.
 The company has allocated a pool of Rs 50000 to be given away as bonus money to the loyal players.

Now the company needs to determine how much bonus money should be given to the players.

Should they base it on the amount of loyalty points? Should it be based on number of games? Or something else?

That’s for you to figure out.



First = 50%
2nd to 10th = (30% Equal Distribution/ player )
11th to 50th = (20% Equal Distribution/ player )

### PART C 

According to me the above formula is correct and vaild and can be used in multiplayer game as per my knwoledge and research.
But in case there might be some other method to calculate loyalty point which can be figured out in convinent way.
